# 台灣各縣市高齡人口與長照資源供需分析

本筆記本依據研究方法，分析民國 113 年台灣各縣市高齡人口結構與長照資源配置情況。

## 研究動機
近年來，台灣快速邁入超高齡社會，高齡人口比例持續攀升。根據內政部人口統計，我國自 109 年起自然增加率轉為負值，顯示出生人口已無法支撐死亡人口，人口老化已成為不可逆的趨勢。
在高齡人口不斷成長的情況下，長期照顧需求急速擴張。政府推動長照 2.0 政策雖大幅擴增服務項目與供給量，但受限於地理分布、人口結構差異、偏鄉交通不便，以及各縣市老化速度不同，全國長照資源仍呈現供需不均與地區差異顯著的現象。部分都市區人口密度高、服務使用率高，容易出現「需求大於供給」；偏鄉或山區則因面積大、機構少、交通距離長，造成「可近性不足」的長照服務缺口。
為了有效配置有限的長照資源，必須透過統計分析，全面檢視各行政區的高齡人口結構、長照設施分布與可能的供需落差，以作為政策推動與資源配置的客觀依據。


**研究方法架構：**
1. 描述性統計
2. 服務密度分析
3. 供需落差分析
4. 交叉比對分析
5. 視覺化統計


In [ ]:
# 導入所需套件
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager
from math import pi
import warnings
import os

warnings.filterwarnings('ignore')

# 設定中文字型
print("正在設定中文字型...")
noto_font_dir = './fonts/Noto_Sans_TC/static'
if os.path.exists(noto_font_dir):
    font_files = {
        'Regular': os.path.join(noto_font_dir, 'NotoSansTC-Regular.ttf'),
        'Bold': os.path.join(noto_font_dir, 'NotoSansTC-Bold.ttf'),
        'Medium': os.path.join(noto_font_dir, 'NotoSansTC-Medium.ttf')
    }
    for weight, font_path in font_files.items():
        if os.path.exists(font_path):
            font_manager.fontManager.addfont(font_path)
    plt.rcParams['font.sans-serif'] = ['Noto Sans TC']
    plt.rcParams['axes.unicode_minus'] = False
    print("✓ 字型設定完成")

sns.set_theme(style='whitegrid', font='Noto Sans TC')
plt.rcParams['figure.figsize'] = (12, 6)


## 資料載入與清理


In [ ]:
# 載入人口資料
df_pop = pd.read_csv('程式用-縣市人口按性別及五齡組.csv', encoding='utf-8')
df_pop.columns = df_pop.columns.str.strip()

# 清理數值欄位
numeric_cols = ['總人口', '65～69', '70～74', '75～79', '80～84', '85～89', '90～94', '95～99', '100以上', '65以下']
for col in numeric_cols:
    if col in df_pop.columns:
        df_pop[col] = df_pop[col].astype(str).str.replace(',', '').str.replace(' ', '').astype(float)

df_pop['區域別'] = df_pop['區域別'].str.strip()

# 計算 65 歲以上人口
age_cols = ['65～69', '70～74', '75～79', '80～84', '85～89', '90～94', '95～99', '100以上']
df_pop['65歲以上'] = df_pop[age_cols].sum(axis=1)
df_pop['老年人口比例(%)'] = (df_pop['65歲以上'] / df_pop['總人口']) * 100

print(f"人口資料: {len(df_pop)} 筆, {df_pop['區域別'].nunique()} 個縣市, 年份 {df_pop['年份'].min()}-{df_pop['年份'].max()}")


In [ ]:
# 載入長照 ABC 據點資料
df_ltc = pd.read_csv('data/長照ABC據點.csv', encoding='utf-8')

# 建立縣市代碼對照表
city_code_mapping = {
    '63000': '台北市', '64000': '高雄市', '65000': '新北市', '66000': '台中市',
    '67000': '台南市', '68000': '桃園市', '10002': '宜蘭縣', '10004': '新竹縣',
    '10005': '苗栗縣', '10007': '彰化縣', '10008': '南投縣', '10009': '雲林縣',
    '10010': '嘉義縣', '10013': '屏東縣', '10014': '台東縣', '10015': '花蓮縣',
    '10016': '澎湖縣', '10017': '基隆市', '10018': '新竹市', '10020': '嘉義市',
    '09007': '連江縣', '09020': '金門縣'
}

df_ltc['縣市代碼'] = df_ltc['縣市'].astype(str).str.strip().str.replace('"', '')
df_ltc['縣市名稱'] = df_ltc['縣市代碼'].map(city_code_mapping)
df_ltc['據點類型'] = df_ltc['O_ABC'].astype(str).str.strip().str.replace('"', '')

# 過濾有效資料
df_ltc_valid = df_ltc[(df_ltc['縣市名稱'].notna()) & (df_ltc['據點類型'].isin(['A', 'B', 'C']))].copy()

print(f"長照據點: {len(df_ltc_valid)} 個")
print(df_ltc_valid['據點類型'].value_counts())


In [ ]:
# 載入土地面積資料
df_area = pd.read_csv('data/程式用-各縣市鄉鎮市區土地面積及人口密度.csv', encoding='utf-8')
df_area['縣市'] = df_area['縣市'].str.strip().str.replace(' ', '').str.replace('　', '')
df_area['土地面積'] = df_area['土地面積'].astype(str).str.replace(',', '').str.replace(' ', '').astype(float)

# 彙整各縣市總土地面積（使用 113 年資料）
df_area_113 = df_area[df_area['年份'] == 113]
city_area = df_area_113.groupby('縣市')['土地面積'].sum().reset_index()
city_area.columns = ['縣市名稱', '土地面積(km²)']

print(f"土地面積資料: {len(city_area)} 個縣市")


In [ ]:
# 合併所有資料
ltc_by_city = df_ltc_valid.groupby(['縣市名稱', '據點類型']).size().unstack(fill_value=0)
ltc_by_city['總據點數'] = ltc_by_city.sum(axis=1)
ltc_by_city = ltc_by_city.reset_index()

df_113 = df_pop[df_pop['年份'] == 113].copy()
df_113['縣市名稱'] = df_113['區域別'].str.replace(' ', '')

df = df_113.merge(ltc_by_city, on='縣市名稱', how='left')
df = df.merge(city_area, on='縣市名稱', how='left')
df[['A', 'B', 'C', '總據點數']] = df[['A', 'B', 'C', '總據點數']].fillna(0)

# 計算人口密度
df['人口密度(人/km²)'] = df['總人口'] / df['土地面積(km²)']

print("✓ 資料合併完成")
print(f"分析對象: 113 年, {len(df)} 個縣市")


---

## 1. 描述性統計

掌握各行政區人口結構與長照資源現況：
- 高齡人口比例（%）
- 實際高齡人口數（= 總人口 × 65+比例）
- 長照單位數量（各類型機構）
- 各區人口密度（人口 ÷ 面積）


In [ ]:
# 1.1 各縣市高齡人口比例與實際高齡人口數
print("=" * 80)
print("各縣市高齡人口統計（113年）")
print("=" * 80)

aging_stats = df[['縣市名稱', '總人口', '65歲以上', '老年人口比例(%)']].sort_values('老年人口比例(%)', ascending=False)
aging_stats['排名'] = range(1, len(aging_stats) + 1)
print(aging_stats[['排名', '縣市名稱', '總人口', '65歲以上', '老年人口比例(%)']].to_string(index=False))

print("\n【敘述統計】")
print(f"平均老年人口比例: {df['老年人口比例(%)'].mean():.2f}%")
print(f"標準差: {df['老年人口比例(%)'].std():.2f}%")
print(f"最高: {df['老年人口比例(%)'].max():.2f}% ({df.loc[df['老年人口比例(%)'].idxmax(), '縣市名稱']})")
print(f"最低: {df['老年人口比例(%)'].min():.2f}% ({df.loc[df['老年人口比例(%)'].idxmin(), '縣市名稱']})")


In [ ]:
# 1.2 各縣市長照單位數量（各類型機構）
print("=" * 80)
print("各縣市長照據點數量統計（113年）")
print("=" * 80)

ltc_stats = df[['縣市名稱', 'A', 'B', 'C', '總據點數']].sort_values('總據點數', ascending=False)
ltc_stats['排名'] = range(1, len(ltc_stats) + 1)
print(ltc_stats[['排名', '縣市名稱', 'A', 'B', 'C', '總據點數']].to_string(index=False))

print("\n【長照據點類型說明】")
print("A 級：社區整合型服務中心（提供多元服務）")
print("B 級：複合型服務中心（提供專業服務）")
print("C 級：巷弄長照站（提供社區照顧）")
print(f"\n【全國總計】A={df['A'].sum():.0f}, B={df['B'].sum():.0f}, C={df['C'].sum():.0f}, 總計={df['總據點數'].sum():.0f}")


In [ ]:
# 1.3 各區人口密度
print("=" * 80)
print("各縣市人口密度（113年）")
print("=" * 80)

density_stats = df[['縣市名稱', '總人口', '土地面積(km²)', '人口密度(人/km²)']].sort_values('人口密度(人/km²)', ascending=False)
density_stats['排名'] = range(1, len(density_stats) + 1)
print(density_stats[['排名', '縣市名稱', '總人口', '土地面積(km²)', '人口密度(人/km²)']].to_string(index=False))

print("\n【敘述統計】")
print(f"平均人口密度: {df['人口密度(人/km²)'].mean():.2f} 人/km²")
print(f"標準差: {df['人口密度(人/km²)'].std():.2f} 人/km²")


---

## 2. 服務密度分析

量化長照資源是否足夠：

**a. 每千位高齡人口的長照資源量**
```
每千位高齡人口的長照機構數 = 長照機構總數 ÷ 高齡人口數 × 1000
```
→ 判斷哪些區資源明顯不足

**b. 每平方公里資源密度**
```
長照機構密度 = 長照機構數 ÷ 面積
```
→ 發現偏鄉面積大、資源不足的情形


In [ ]:
# 2a. 每千位高齡人口的長照機構數
df['每千位高齡人口據點數'] = (df['總據點數'] / df['65歲以上']) * 1000

print("=" * 80)
print("每千位高齡人口的長照據點數（113年）")
print("=" * 80)
print("公式: 長照據點總數 ÷ 65歲以上人口 × 1000\n")

service_elderly = df[['縣市名稱', '65歲以上', '總據點數', '每千位高齡人口據點數']].sort_values('每千位高齡人口據點數', ascending=False)
service_elderly['排名'] = range(1, len(service_elderly) + 1)
print(service_elderly[['排名', '縣市名稱', '65歲以上', '總據點數', '每千位高齡人口據點數']].to_string(index=False))

avg_service = df['每千位高齡人口據點數'].mean()
print(f"\n【敘述統計】")
print(f"平均: {avg_service:.2f} 個/千人")
print(f"標準差: {df['每千位高齡人口據點數'].std():.2f}")
print(f"→ 低於平均的縣市可能資源不足")


In [ ]:
# 2b. 每平方公里資源密度
df['每平方公里據點數'] = df['總據點數'] / df['土地面積(km²)']

print("=" * 80)
print("每平方公里的長照據點數（113年）")
print("=" * 80)
print("公式: 長照據點數 ÷ 土地面積(km²)\n")

service_area = df[['縣市名稱', '土地面積(km²)', '總據點數', '每平方公里據點數']].sort_values('每平方公里據點數', ascending=False)
service_area['排名'] = range(1, len(service_area) + 1)
print(service_area[['排名', '縣市名稱', '土地面積(km²)', '總據點數', '每平方公里據點數']].to_string(index=False))

print(f"\n【敘述統計】")
print(f"平均: {df['每平方公里據點數'].mean():.4f} 個/km²")
print(f"→ 偏鄉面積大但據點密度低，可能有可近性問題")


---

## 3. 供需落差分析

評估各區長照機構是否「不足」或「過量」：
```
服務缺口 = 所需據點數 − 現有據點數
```
找出供需失衡的區域，用於政策建議方向。


In [ ]:
# 3.1 計算服務缺口
# 以全國平均「每千位高齡人口據點數」作為合理服務標準
avg_service_rate = df['每千位高齡人口據點數'].mean()

# 計算各縣市所需據點數
df['所需據點數'] = (df['65歲以上'] * avg_service_rate / 1000).round(0)
df['服務缺口'] = df['所需據點數'] - df['總據點數']
df['缺口比例(%)'] = (df['服務缺口'] / df['所需據點數'] * 100).round(2)

print("=" * 80)
print("長照服務供需落差分析（113年）")
print("=" * 80)
print(f"合理服務標準: 每千位高齡人口 {avg_service_rate:.2f} 個據點（全國平均）\n")

gap_analysis = df[['縣市名稱', '65歲以上', '總據點數', '所需據點數', '服務缺口', '缺口比例(%)']].sort_values('服務缺口', ascending=False)
print(gap_analysis.to_string(index=False))

print("\n【解讀】")
print("• 服務缺口 > 0：據點不足，需增設")
print("• 服務缺口 < 0：據點充足或過量")


In [ ]:
# 3.2 供需失衡區域識別
print("=" * 80)
print("供需失衡區域識別")
print("=" * 80)

# 資源不足區域
shortage = df[df['服務缺口'] > 0].sort_values('服務缺口', ascending=False)
print(f"\n【資源不足區域】共 {len(shortage)} 個縣市：")
if len(shortage) > 0:
    print(shortage[['縣市名稱', '服務缺口', '缺口比例(%)']].to_string(index=False))
else:
    print("無")

# 資源充足區域
surplus = df[df['服務缺口'] < 0].sort_values('服務缺口')
print(f"\n【資源充足區域】共 {len(surplus)} 個縣市：")
if len(surplus) > 0:
    print(surplus[['縣市名稱', '服務缺口', '缺口比例(%)']].to_string(index=False))


---

## 4. 交叉比對分析

把不同資料疊合，找出「資源不足熱點」。

找出「高齡比例高但資源少」的分區，作為優先設置長照據點的依據。


In [ ]:
# 4.1 建立優先順序評估
# 計算標準化分數
df['老化程度Z'] = (df['老年人口比例(%)'] - df['老年人口比例(%)'].mean()) / df['老年人口比例(%)'].std()
df['資源充足度Z'] = (df['每千位高齡人口據點數'] - df['每千位高齡人口據點數'].mean()) / df['每千位高齡人口據點數'].std()

# 優先設置分數 = 老化程度高 + 資源充足度低
df['優先設置分數'] = df['老化程度Z'] - df['資源充足度Z']

print("=" * 80)
print("長照據點優先設置建議排名")
print("=" * 80)
print("評估標準: 優先設置分數 = 老化程度(標準化) - 資源充足度(標準化)")
print("分數越高 = 老化程度高且資源不足，應優先設置\n")

priority = df[['縣市名稱', '老年人口比例(%)', '每千位高齡人口據點數', '優先設置分數']].sort_values('優先設置分數', ascending=False)
priority['優先順序'] = range(1, len(priority) + 1)
print(priority[['優先順序', '縣市名稱', '老年人口比例(%)', '每千位高齡人口據點數', '優先設置分數']].to_string(index=False))


In [ ]:
# 4.2 識別資源不足熱點
# 定義熱點：老化程度高於平均 且 資源充足度低於平均
avg_aging = df['老年人口比例(%)'].mean()
avg_resource = df['每千位高齡人口據點數'].mean()

hotspots = df[
    (df['老年人口比例(%)'] > avg_aging) & 
    (df['每千位高齡人口據點數'] < avg_resource)
].sort_values('優先設置分數', ascending=False)

print("=" * 80)
print("資源不足熱點（高齡比例高 + 資源少）")
print("=" * 80)
print(f"判斷標準：老年人口比例 > {avg_aging:.2f}% 且 每千位高齡人口據點數 < {avg_resource:.2f}\n")

if len(hotspots) > 0:
    print(f"共 {len(hotspots)} 個熱點區域：")
    print(hotspots[['縣市名稱', '老年人口比例(%)', '每千位高齡人口據點數', '服務缺口']].to_string(index=False))
    print("\n→ 建議優先於上述區域增設長照據點")
else:
    print("無符合條件的熱點區域")


---

## 5. 視覺化統計

用圖表呈現差異、趨勢、缺口：
- 長條圖
- 散佈圖
- 雷達圖


In [ ]:
# 5.1 長條圖：各縣市老年人口比例
fig, ax = plt.subplots(figsize=(14, 8))

data = df.sort_values('老年人口比例(%)', ascending=True)
colors = ['#d62728' if x > avg_aging else '#1f77b4' for x in data['老年人口比例(%)']]

bars = ax.barh(data['縣市名稱'], data['老年人口比例(%)'], color=colors)
ax.axvline(x=avg_aging, color='red', linestyle='--', linewidth=2, label=f'全國平均 ({avg_aging:.2f}%)')

ax.set_xlabel('老年人口比例 (%)', fontsize=12)
ax.set_title('各縣市老年人口比例（113年）', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3, axis='x')

for bar, val in zip(bars, data['老年人口比例(%)']):
    ax.text(val + 0.2, bar.get_y() + bar.get_height()/2, f'{val:.1f}%', va='center', fontsize=9)

plt.tight_layout()
plt.show()


In [ ]:
# 5.2 長條圖：各縣市服務缺口
fig, ax = plt.subplots(figsize=(14, 8))

data = df.sort_values('服務缺口', ascending=True)
colors = ['#d62728' if x > 0 else '#2ca02c' for x in data['服務缺口']]

bars = ax.barh(data['縣市名稱'], data['服務缺口'], color=colors)
ax.axvline(x=0, color='black', linestyle='-', linewidth=1)

ax.set_xlabel('服務缺口（據點數）', fontsize=12)
ax.set_title('各縣市長照服務缺口（113年）', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

for bar, val in zip(bars, data['服務缺口']):
    offset = 5 if val >= 0 else -5
    ha = 'left' if val >= 0 else 'right'
    ax.text(val + offset, bar.get_y() + bar.get_height()/2, f'{val:.0f}', va='center', ha=ha, fontsize=9)

from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='#d62728', label='資源不足'),
                   Patch(facecolor='#2ca02c', label='資源充足')]
ax.legend(handles=legend_elements, loc='lower right')

plt.tight_layout()
plt.show()


In [ ]:
# 5.3 散佈圖：老年人口比例 vs 每千位高齡人口據點數
fig, ax = plt.subplots(figsize=(12, 8))

scatter = ax.scatter(df['老年人口比例(%)'], df['每千位高齡人口據點數'], 
                     s=df['65歲以上']/5000, alpha=0.6, c=df['服務缺口'], 
                     cmap='RdYlGn_r', edgecolors='black', linewidth=0.5)

# 添加縣市標籤
for idx, row in df.iterrows():
    ax.annotate(row['縣市名稱'], (row['老年人口比例(%)'], row['每千位高齡人口據點數']),
                xytext=(5, 5), textcoords='offset points', fontsize=8)

# 添加平均線
ax.axhline(y=avg_resource, color='blue', linestyle='--', alpha=0.5, label=f'資源平均 ({avg_resource:.2f})')
ax.axvline(x=avg_aging, color='red', linestyle='--', alpha=0.5, label=f'老化平均 ({avg_aging:.2f}%)')

# 標註四個象限
xlim = ax.get_xlim()
ylim = ax.get_ylim()
ax.text(xlim[1]-1, ylim[1]-0.5, '高老化\n高資源', fontsize=10, ha='right', color='green')
ax.text(xlim[0]+0.5, ylim[1]-0.5, '低老化\n高資源', fontsize=10, ha='left', color='blue')
ax.text(xlim[1]-1, ylim[0]+0.5, '高老化\n低資源\n(熱點)', fontsize=10, ha='right', color='red', fontweight='bold')
ax.text(xlim[0]+0.5, ylim[0]+0.5, '低老化\n低資源', fontsize=10, ha='left', color='gray')

ax.set_xlabel('老年人口比例 (%)', fontsize=12)
ax.set_ylabel('每千位高齡人口據點數', fontsize=12)
ax.set_title('老年人口比例 vs 長照資源密度（氣泡大小=高齡人口數）', fontsize=14, fontweight='bold')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

cbar = plt.colorbar(scatter)
cbar.set_label('服務缺口')

plt.tight_layout()
plt.show()


In [ ]:
# 5.4 雷達圖：比較代表性縣市多維度指標
# 選取代表性縣市
selected_cities = [
    df.loc[df['老年人口比例(%)'].idxmax(), '縣市名稱'],  # 老化最高
    df.loc[df['老年人口比例(%)'].idxmin(), '縣市名稱'],  # 老化最低
    df.loc[df['每千位高齡人口據點數'].idxmax(), '縣市名稱'],  # 資源最多
    df.loc[df['每千位高齡人口據點數'].idxmin(), '縣市名稱'],  # 資源最少
    df.loc[df['總人口'].idxmax(), '縣市名稱'],  # 人口最多
]
selected_cities = list(dict.fromkeys(selected_cities))  # 去重保序

# 準備雷達圖資料
categories = ['老年人口比例', '每千位高齡人口據點數', '每平方公里據點數', '人口密度']
N = len(categories)

# 標準化各指標到 0-1 範圍
df_radar = df[df['縣市名稱'].isin(selected_cities)].copy()
for col, orig in zip(['老年人口比例(%)_norm', '每千位高齡人口據點數_norm', '每平方公里據點數_norm', '人口密度(人/km²)_norm'],
                     ['老年人口比例(%)', '每千位高齡人口據點數', '每平方公里據點數', '人口密度(人/km²)']):
    min_val = df[orig].min()
    max_val = df[orig].max()
    df_radar[col] = (df_radar[orig] - min_val) / (max_val - min_val)

# 繪製雷達圖
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))

angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

colors = plt.cm.Set2(np.linspace(0, 1, len(selected_cities)))

for idx, city in enumerate(selected_cities):
    city_data = df_radar[df_radar['縣市名稱'] == city]
    values = [
        city_data['老年人口比例(%)_norm'].values[0],
        city_data['每千位高齡人口據點數_norm'].values[0],
        city_data['每平方公里據點數_norm'].values[0],
        city_data['人口密度(人/km²)_norm'].values[0]
    ]
    values += values[:1]
    
    ax.plot(angles, values, 'o-', linewidth=2, label=city, color=colors[idx])
    ax.fill(angles, values, alpha=0.1, color=colors[idx])

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, fontsize=11)
ax.set_title('代表性縣市多維度指標比較（雷達圖）', fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))

plt.tight_layout()
plt.show()


---

## 結論與建議


In [ ]:
print("=" * 80)
print("研究結論與政策建議")
print("=" * 80)

print("\n【一、整體概況】")
print(f"分析年份: 113 年")
print(f"縣市數量: {len(df)} 個")
print(f"全國高齡人口: {df['65歲以上'].sum():,.0f} 人")
print(f"全國平均老年人口比例: {avg_aging:.2f}%")
print(f"全國長照據點總數: {df['總據點數'].sum():.0f} 個")
print(f"全國平均每千位高齡人口據點數: {avg_resource:.2f} 個")

print("\n【二、老化程度分析】")
top3_aging = df.nlargest(3, '老年人口比例(%)')[['縣市名稱', '老年人口比例(%)']]
print("老化程度最高前三名：")
for _, row in top3_aging.iterrows():
    print(f"  • {row['縣市名稱']}: {row['老年人口比例(%)']:.2f}%")

print("\n【三、資源不足熱點】")
if len(hotspots) > 0:
    print(f"共 {len(hotspots)} 個區域需優先關注：")
    for _, row in hotspots.iterrows():
        print(f"  • {row['縣市名稱']}: 老化率 {row['老年人口比例(%)']:.2f}%, 缺口 {row['服務缺口']:.0f} 個據點")
else:
    print("無明顯熱點區域")

print("\n【四、政策建議】")
priority_top5 = df.nlargest(5, '優先設置分數')[['縣市名稱', '服務缺口']]
print("建議優先設置長照據點的區域：")
for i, (_, row) in enumerate(priority_top5.iterrows(), 1):
    gap = max(0, row['服務缺口'])
    print(f"  {i}. {row['縣市名稱']}（建議增設 {gap:.0f} 個據點）")

print("\n" + "=" * 80)
